In [1]:
import tensorflow as tf
import numpy as np
import random, os, glob, time
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd

In [2]:
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)

# Data and image parameters
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
TRAIN_DIR = 'data/chest_xray/chest_xray/train'
TEST_DIR  = 'data/chest_xray/chest_xray/test'
# Assume training folder has one subfolder per class (e.g., NORMAL, PNEUMONIA)
NUM_CLASSES = len([d for d in os.listdir(TRAIN_DIR) if os.path.isdir(os.path.join(TRAIN_DIR, d))])
# For episodic (few–shot) training:
K_SHOT  = 5  # number of support images per class per episode
Q_QUERY = 5  # number of query images per class per episode

print(f"Found {NUM_CLASSES} classes in training data.")

Found 2 classes in training data.


In [3]:
def load_data_by_class(data_dir):
    """
    Loads file paths for each class into a dictionary.
    """
    data = {}
    for cls in os.listdir(data_dir):
        full_dir = os.path.join(data_dir, cls)
        if os.path.isdir(full_dir):
            files = glob.glob(os.path.join(full_dir, '*'))
            data[cls] = files
    return data

# Load training and test data by class (for episodic training)
train_data = load_data_by_class(TRAIN_DIR)
test_data  = load_data_by_class(TEST_DIR)

def generate_episode(data, num_classes=NUM_CLASSES, k_shot=K_SHOT, q_query=Q_QUERY):
    """
    Generate one episode:
      - Randomly sample 'num_classes' from available classes.
      - For each, randomly sample k_shot support images and q_query query images.
    Returns:
      support_images, support_labels, query_images, query_labels.
    Labels are one–hot encoded (order is per–episode).
    """
    support_images, support_labels = [], []
    query_images, query_labels = [], []
    selected_classes = random.sample(list(data.keys()), num_classes)
    for i, cls in enumerate(selected_classes):
        images = random.sample(data[cls], k_shot + q_query)
        support_files = images[:k_shot]
        query_files   = images[k_shot:]
        for img_path in support_files:
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img = tf.keras.preprocessing.image.img_to_array(img)
            support_images.append(img)
            label = np.zeros(num_classes)
            label[i] = 1
            support_labels.append(label)
        for img_path in query_files:
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img = tf.keras.preprocessing.image.img_to_array(img)
            query_images.append(img)
            label = np.zeros(num_classes)
            label[i] = 1
            query_labels.append(label)
    # Normalize images:
    support_images = np.array(support_images) / 255.0
    query_images   = np.array(query_images)   / 255.0
    return (support_images, 
        np.array(support_labels, dtype=np.float32), 
        query_images, 
        np.array(query_labels, dtype=np.float32))

In [4]:
def build_original_model():
    base_model = VGG16(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    # Freeze base model layers
    for layer in base_model.layers:
        layer.trainable = False
    x = Flatten()(base_model.output)
    output = Dense(NUM_CLASSES, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=output)

original_model = build_original_model()
original_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
original_model.summary()

# Train using ImageDataGenerator (note: for demo, epochs are kept low)
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen  = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=4, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
    TEST_DIR, target_size=(IMG_HEIGHT, IMG_WIDTH), batch_size=4, class_mode='categorical')

print("\nTraining Original Model...")
t0 = time.time()
original_history = original_model.fit(train_generator,
                                      validation_data=test_generator,
                                      epochs=5,
                                      steps_per_epoch=len(train_generator),
                                      validation_steps=len(test_generator))
original_training_time = time.time() - t0

# Evaluate original model on the test set:
original_eval = original_model.evaluate(test_generator, steps=len(test_generator))
print("\nOriginal Model Evaluation:", original_eval)
y_pred = np.argmax(original_model.predict(test_generator, steps=len(test_generator)), axis=1)
y_true = test_generator.classes
cm_original = confusion_matrix(y_true, y_pred)
report_original = classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys()))
print("Confusion Matrix (Original):\n", cm_original)
print("Classification Report (Original):\n", report_original)

# Save the original model:
original_model.save('models/original_model.h5')
print("Saved original_model.h5")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Found 2653 images belonging to 2 classes.
Found 624 images belonging to 2 classes.

Training Original Model...


/Users/bobbytomlinson/Desktop/Coding/Personal/new/FewShotLearningForPneumonia/pneuFSLvenv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
664/664 ━━━━━━━━━━━━━━━━━━━━ 212s 318ms/step - accuracy: 0.8606 - loss: 0.4809 - val_accuracy: 0.9087 - val_loss: 0.3727
Epoch 2/5
145/664 ━━━━━━━━━━━━━━━━━━━━ 2:21 272ms/step - accuracy: 0.9189 - loss: 0.2941

KeyboardInterrupt: 

In [ ]:
def build_backbone():
    """
    Build a VGG16–based backbone that outputs an embedding.
    (Note: base layers are frozen.)
    """
    vgg = VGG16(input_shape=IMG_SHAPE, weights='imagenet', include_top=False)
    for layer in vgg.layers:
        layer.trainable = False
    x = Flatten()(vgg.output)
    embedding = Dense(128, activation='relu', name='embedding')(x)
    return Model(inputs=vgg.input, outputs=embedding, name='Backbone')

backbone_fsl = build_backbone()
backbone_fsl.summary()

Model: "Backbone"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Dense)               │ (None, 128)            │     3,211,392 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,926,080 (68.38 MB)

 Trainable params: 3,211,392 (12.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
def get_fresh_backbone():
    return build_backbone()

In [ ]:
# -----------------------------------------------------------------------------
# 5c. Prototypical Networks
# -----------------------------------------------------------------------------

proto_backbone = get_fresh_backbone()


def prototypical_network_prediction(embedding_model, support_imgs, support_lbls, query_imgs, num_classes=NUM_CLASSES):
    support_embeddings = embedding_model(support_imgs)
    query_embeddings   = embedding_model(query_imgs)
    prototypes = []
    for c in range(num_classes):
        class_mask = tf.equal(tf.argmax(support_lbls, axis=1), c)
        class_embeddings = tf.boolean_mask(support_embeddings, class_mask)
        proto = tf.reduce_mean(class_embeddings, axis=0)
        prototypes.append(proto)
    prototypes = tf.stack(prototypes)  # shape: (num_classes, emb_dim)
    distances = tf.reduce_sum(tf.square(tf.expand_dims(query_embeddings, axis=1) - prototypes), axis=2)
    logits = -distances  # negative Euclidean distance
    predictions = tf.nn.softmax(logits, axis=1)
    return predictions

proto_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def proto_train_step(embedding_model, support_imgs, support_lbls, query_imgs, query_lbls):
    with tf.GradientTape() as tape:
        preds = prototypical_network_prediction(embedding_model, support_imgs, support_lbls, query_imgs)
        loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(query_lbls, preds))
    grads = tape.gradient(loss, embedding_model.trainable_variables)
    proto_optimizer.apply_gradients(zip(grads, embedding_model.trainable_variables))
    return loss

# Train Prototypical Networks episodically
proto_episodes = 50
print("\nTraining Prototypical Networks...")
t0 = time.time()
for episode in range(proto_episodes):
    s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(train_data)
    loss = proto_train_step(proto_backbone, s_imgs, s_lbls, q_imgs, q_lbls)
    if episode % 10 == 0:
        print(f"Prototypical Networks Episode {episode}: Loss = {loss.numpy():.4f}")
proto_training_time = time.time() - t0

def evaluate_proto(embedding_model, data, num_episodes=20):
    total_correct = 0
    total_samples = 0
    for _ in range(num_episodes):
        s_imgs, s_lbls, q_imgs, q_lbls = generate_episode(data)
        preds = prototypical_network_prediction(embedding_model, s_imgs, s_lbls, q_imgs)
        pred_classes = tf.argmax(preds, axis=1)
        true_classes = tf.argmax(q_lbls, axis=1)
        total_correct += np.sum(pred_classes.numpy() == true_classes.numpy())
        total_samples += q_imgs.shape[0]
    return total_correct / total_samples

proto_accuracy = evaluate_proto(proto_backbone, test_data)
print("Prototypical Networks Evaluation Accuracy:", proto_accuracy)

proto_backbone.save('models/proto_backbone.h5')
print("Saved proto_backbone.h5")


Training Prototypical Networks...
Prototypical Networks Episode 0: Loss = 0.4226
Prototypical Networks Episode 10: Loss = 7.4394
Prototypical Networks Episode 20: Loss = 2.8208
Prototypical Networks Episode 30: Loss = 0.0000
Prototypical Networks Episode 40: Loss = 2.4652


Prototypical Networks Evaluation Accuracy: 0.885
Saved proto_backbone.h5


In [ ]:
# -----------------------------------------------------------------------------
# 6. Comparative Results Summary
# -----------------------------------------------------------------------------
results = {
    "Original Model": {
        "Accuracy": original_eval[1],
        "Training Time (s)": original_training_time,
        "Confusion Matrix": cm_original,
        "Classification Report": report_original,
    },
    "Prototypical Networks": {
        "Accuracy": proto_accuracy,
        "Training Time (s)": proto_training_time,
    }
}

print("\n=== Comparative Results ===")
for method, metrics in results.items():
    print(f"\nMethod: {method}")
    for key, value in metrics.items():
        print(f"  {key}: {value}")


=== Comparative Results ===

Method: Original Model
  Accuracy: 0.8894230723381042
  Training Time (s): 1896.089072227478
  Confusion Matrix: [[ 61 173]
 [108 282]]
  Classification Report:               precision    recall  f1-score   support

      NORMAL       0.36      0.26      0.30       234
   PNEUMONIA       0.62      0.72      0.67       390

    accuracy                           0.55       624
   macro avg       0.49      0.49      0.49       624
weighted avg       0.52      0.55      0.53       624


Method: MAML
  Accuracy: 0.505
  Training Time (s): 67.7479157447815

Method: Matching Networks
  Accuracy: 0.655
  Training Time (s): 70.79149127006531

Method: Prototypical Networks
  Accuracy: 0.885
  Training Time (s): 67.75871014595032

Method: Relation Networks
  Accuracy: 0.59
  Training Time (s): 68.0609540939331


In [ ]:
results = {
    "Original Model": {
        "Accuracy": original_eval[1],
        "Training Time (s)": original_training_time,
        "Confusion Matrix": str(cm_original),
        "Classification Report": report_original,
    },
    "Prototypical Networks": {
        "Accuracy": proto_accuracy,
        "Training Time (s)": proto_training_time,
    }
}

# Convert dictionary to DataFrame and export to CSV:
rows = []
for method, metrics in results.items():
    row = {"Method": method}
    for key, value in metrics.items():
        row[key] = value
    rows.append(row)

df = pd.DataFrame(rows)
csv_filename = "fsl_comparison_metrics.csv"
df.to_csv(csv_filename, index=False)
print(f"\nExported metrics to {csv_filename}")


Exported metrics to fsl_comparison_metrics.csv
